# $\textit{Cancelación activa de ruido}$
$\textit{By }$

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as sp
import scipy.io as io
#import helper as hp
import importlib
from numpy.fft import fft, rfft
from numpy.fft import fftshift, fftfreq, rfftfreq
from IPython.display import Audio
from statsmodels.tsa.stattools import pacf, acf
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [10]:
data = io.loadmat("./MATLAB_raw/PANDAR_load_and_process_raw.m")

ValueError: Unknown mat file type, version 100, 105